## 병원 데이터 처리

In [183]:
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import korean

In [184]:
hospital_data = pd.read_csv('./data/서울시 병원 인허가 정보.csv', encoding="cp949")
hospital_data.columns

Index(['개방자치단체코드', '관리번호', '인허가일자', '인허가취소일자', '영업상태코드', '영업상태명', '상세영업상태코드',
       '상세영업상태명', '폐업일자', '휴업시작일자', '휴업종료일자', '재개업일자', '전화번호', '소재지면적',
       '소재지우편번호', '지번주소', '도로명주소', '도로명우편번호', '사업장명', '최종수정일자', '데이터갱신구분',
       '데이터갱신일자', '업태구분명', '좌표정보(X)', '좌표정보(Y)'],
      dtype='object')

In [185]:
hospital_df = hospital_data[['상세영업상태명', '지번주소', '도로명주소', '사업장명']]
hospital_df

,상세영업상태명,지번주소,도로명주소,사업장명
0,영업중,서울특별시 강남구 논현동 5-4,"서울특별시 강남구 도산대로 138, 13층 (논현동)",리페이스치과병원
1,영업중,"서울특별시 송파구 문정동 634 가든파이브라이프 10층 L-10001~4, L-1...","서울특별시 송파구 충민로 66, 가든파이브라이프 10층 L-10001~4,L-100...",잠실자생한방병원
2,영업중,NaN,서울특별시 강동구 천호대로 1044 (성내동),리더스병원
3,영업중,서울특별시 강동구 길동 412-15,"서울특별시 강동구 양재대로 1463, 5~15층 (길동)",강동아너스힐병원
4,영업중,NaN,"서울특별시 송파구 법원로11길 32, 지하2~1층, 지상5~11층,13층 (문정동)",대전대학교서울한방병원
...,...,...,...,...
897,영업중,서울특별시 중랑구 망우본동 227번지,서울특별시 중랑구 양원역로 38 (망우동),서울특별시 북부병원
898,영업중,서울특별시 동작구 신대방동 425번지,서울특별시 동작구 보라매로5길 20 (신대방동),서울특별시보라매병원
899,영업중,서울특별시 강북구 수유동 45번지 7호,"서울특별시 강북구 도봉로 301 (수유동, 대한병원)",의료법인성화의료재단 대한병원
900,영업중,NaN,"서울특별시 광진구 동일로 82-1, 생생한방병원 지하1~5층 (자양동)",경희생생한방병원


In [186]:
# ================================
# 노원구 소재 & 영업중인 병원 필터링
# ================================

# 지번주소와 도로명주소 중 값이 있는 것을 하나의 '주소' 컬럼으로 결합
# (둘 중 하나가 비어있을 수 있으므로 fillna('') 사용)
hospital_df['주소'] = (
    hospital_df['지번주소'].fillna('') + ' ' +
    hospital_df['도로명주소'].fillna('')
)

# 주소에 '노원구'가 포함되어 있고,
# 병원 영업 상태가 '영업중'인 데이터만 필터링
nowon_hospital_df = hospital_df[
    hospital_df['주소'].str.contains('노원구', na=False) &
    hospital_df['상세영업상태명'].str.contains('영업중', na=False)
]

# -------------------------------
# 주소 문자열에서 자치구 이름 추출
# -------------------------------
def nowon_gu(address):
    """
    주소 문자열을 공백 기준으로 분리한 후,
    '구'로 끝나는 단어를 찾아 자치구 이름을 반환
    """
    parts = address.split()
    for part in parts:
        if part.endswith('구'):
            return part
    return None  # 자치구를 찾지 못한 경우

# 추출한 자치구 정보를 새로운 컬럼으로 추가
nowon_hospital_df['자치구'] = nowon_hospital_df['주소'].apply(nowon_gu)

# 최종 노원구 병원 데이터프레임 확인
nowon_hospital_df


C:\Users\User\AppData\Local\Temp\ipykernel_11648\121437963.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_df['주소'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_11648\121437963.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nowon_hospital_df['자치구'] = nowon_hospital_df['주소'].apply(nowon_gu)


,상세영업상태명,지번주소,도로명주소,사업장명,주소,자치구
301,영업중,NaN,"서울특별시 노원구 동일로 987, 태릉마이크로병원건물 2,5,6,7층 (공릉동)",모아한방병원,"서울특별시 노원구 동일로 987, 태릉마이크로병원건물 2,5,6,7층 (공릉동)",노원구
310,영업중,NaN,"서울특별시 노원구 노해로 488, 근호빌딩 5층 502호 (상계동)",샤인치과교정과 치과병원,"서울특별시 노원구 노해로 488, 근호빌딩 5층 502호 (상계동)",노원구
311,영업중,서울특별시 노원구 상계동 301-6 파로스병원,서울특별시 노원구 노원로 416(상계동),새힘병원,서울특별시 노원구 상계동 301-6 파로스병원 서울특별시 노원구 노원로 416(상계동),노원구
312,영업중,NaN,"서울특별시 노원구 동일로 1342, 상계백병원 (상계동)",인제대학교 상계백병원,"서울특별시 노원구 동일로 1342, 상계백병원 (상계동)",노원구
313,영업중,NaN,"서울특별시 노원구 노원로 75, 한국원자력의학원 (공릉동)",한국원자력의학원원자력병원,"서울특별시 노원구 노원로 75, 한국원자력의학원 (공릉동)",노원구
314,영업중,NaN,"서울특별시 노원구 노원로 461, 대감프라자빌딩 3,4,5,6층 (상계동)",그레이스힐 요양병원,"서울특별시 노원구 노원로 461, 대감프라자빌딩 3,4,5,6층 (상계동)",노원구
315,영업중,NaN,"서울특별시 노원구 노해로81길 5-6, 노원스퀘어 5-7층 (상계동)",노원 강북예치과병원,"서울특별시 노원구 노해로81길 5-6, 노원스퀘어 5-7층 (상계동)",노원구
316,영업중,서울특별시 노원구 상계동 711번지 12호 9층~11층,"서울특별시 노원구 동일로 1419, 9층~11층 (상계동)",선의요양병원,서울특별시 노원구 상계동 711번지 12호 9층~11층 서울특별시 노원구 동일로 1...,노원구
317,영업중,"서울특별시 노원구 상계동 373번지 71호 , 상계동 373-45 , 상계동 373...","서울특별시 노원구 노원로 448 (상계동, 메디아이여성병원)",메디아이여성병원,"서울특별시 노원구 상계동 373번지 71호 , 상계동 373-45 , 상계동 373...",노원구
318,영업중,서울특별시 노원구 공릉동 680-10,"서울특별시 노원구 동일로 989, 1~9층 (공릉동)",방병원,"서울특별시 노원구 공릉동 680-10 서울특별시 노원구 동일로 989, 1~9층 (...",노원구


In [187]:
nowon_hospital_df_count = nowon_hospital_df['자치구'].value_counts()
nowon_hospital_df_count

자치구
노원구    22
Name: count, dtype: int64

In [188]:
def nowon_dong_from_addresses(jibun, doromyeong):
    """
    지번주소와 도로명주소를 이용해 법정동(○○동)을 추출하는 함수
    - 두 주소 중 하나라도 동 정보가 있으면 반환
    - 찾지 못하면 None 반환
    """

    # 1. 입력값이 문자열이 아닐 경우 오류 방지를 위해 빈 문자열로 변환
    jibun = jibun if isinstance(jibun, str) else ''
    doromyeong = doromyeong if isinstance(doromyeong, str) else ''
    
    # 2. 지번주소 → 도로명주소 순서로 동 추출 시도
    for address in [jibun, doromyeong]:

        # 2-1. 주소에 괄호가 있는 경우 (예: "... (상계동, 노원구)")
        #       괄호 안의 내용을 우선적으로 확인
        if '(' in address:
            inside = address.split('(')[1]

            # 닫는 괄호가 있으면 그 이전까지만 사용
            if ')' in inside:
                inside = inside.split(')')[0]

            # 쉼표 앞까지만 추출 (동 정보만 남기기 위함)
            inside = inside.split(',')[0].strip()

            # 괄호 안에 '동'이 포함되어 있으면 해당 값 반환
            if '동' in inside:
                return inside

        # 2-2. 괄호가 없는 주소인 경우
        #       공백 단위로 나누어 '동'으로 끝나는 단어 탐색
        else:
            for part in address.split():
                part = part.strip()
                if '동' in part:
                    return part

    # 3. 지번주소와 도로명주소 어디에서도 동을 찾지 못한 경우
    return None


# 각 병원의 지번주소와 도로명주소를 이용해 법정동 컬럼 생성
nowon_hospital_df['법정동'] = nowon_hospital_df.apply(
    lambda row: nowon_dong_from_addresses(row['지번주소'], row['도로명주소']),
    axis=1
)

# 법정동이 추가된 노원구 병원 데이터 확인
nowon_hospital_df


C:\Users\User\AppData\Local\Temp\ipykernel_11648\628734889.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nowon_hospital_df['법정동'] = nowon_hospital_df.apply(


,상세영업상태명,지번주소,도로명주소,사업장명,주소,자치구,법정동
301,영업중,NaN,"서울특별시 노원구 동일로 987, 태릉마이크로병원건물 2,5,6,7층 (공릉동)",모아한방병원,"서울특별시 노원구 동일로 987, 태릉마이크로병원건물 2,5,6,7층 (공릉동)",노원구,공릉동
310,영업중,NaN,"서울특별시 노원구 노해로 488, 근호빌딩 5층 502호 (상계동)",샤인치과교정과 치과병원,"서울특별시 노원구 노해로 488, 근호빌딩 5층 502호 (상계동)",노원구,상계동
311,영업중,서울특별시 노원구 상계동 301-6 파로스병원,서울특별시 노원구 노원로 416(상계동),새힘병원,서울특별시 노원구 상계동 301-6 파로스병원 서울특별시 노원구 노원로 416(상계동),노원구,상계동
312,영업중,NaN,"서울특별시 노원구 동일로 1342, 상계백병원 (상계동)",인제대학교 상계백병원,"서울특별시 노원구 동일로 1342, 상계백병원 (상계동)",노원구,상계동
313,영업중,NaN,"서울특별시 노원구 노원로 75, 한국원자력의학원 (공릉동)",한국원자력의학원원자력병원,"서울특별시 노원구 노원로 75, 한국원자력의학원 (공릉동)",노원구,공릉동
314,영업중,NaN,"서울특별시 노원구 노원로 461, 대감프라자빌딩 3,4,5,6층 (상계동)",그레이스힐 요양병원,"서울특별시 노원구 노원로 461, 대감프라자빌딩 3,4,5,6층 (상계동)",노원구,상계동
315,영업중,NaN,"서울특별시 노원구 노해로81길 5-6, 노원스퀘어 5-7층 (상계동)",노원 강북예치과병원,"서울특별시 노원구 노해로81길 5-6, 노원스퀘어 5-7층 (상계동)",노원구,상계동
316,영업중,서울특별시 노원구 상계동 711번지 12호 9층~11층,"서울특별시 노원구 동일로 1419, 9층~11층 (상계동)",선의요양병원,서울특별시 노원구 상계동 711번지 12호 9층~11층 서울특별시 노원구 동일로 1...,노원구,상계동
317,영업중,"서울특별시 노원구 상계동 373번지 71호 , 상계동 373-45 , 상계동 373...","서울특별시 노원구 노원로 448 (상계동, 메디아이여성병원)",메디아이여성병원,"서울특별시 노원구 상계동 373번지 71호 , 상계동 373-45 , 상계동 373...",노원구,상계동
318,영업중,서울특별시 노원구 공릉동 680-10,"서울특별시 노원구 동일로 989, 1~9층 (공릉동)",방병원,"서울특별시 노원구 공릉동 680-10 서울특별시 노원구 동일로 989, 1~9층 (...",노원구,공릉동


In [189]:
nowon_hospital_df['법정동'].unique()

array(['공릉동', '상계동', '하계동', '중계동', '월계동'], dtype=object)

In [190]:
# ================================
# 노원구 병원의 동별 개수 집계
# ================================

# 노원구 병원 데이터 중 자치구와 법정동 정보만 추출
total_nowon_hospital_df = nowon_hospital_df[['자치구', '법정동']]

# 법정동 기준으로 병원 개수 집계
nowon_hospital_df_count = nowon_hospital_df['법정동'].value_counts()

# 집계 결과를 데이터프레임으로 변환
nowon_df = pd.DataFrame(nowon_hospital_df_count)

# 컬럼명 정리 (병원 개수 컬럼 이름 변경)
nowon_df.rename(columns={'count': '병원수'}, inplace=True)

# 인덱스를 컬럼으로 변환하여 정리된 형태로 변경
nowon_df_reset = nowon_df.reset_index()

# 보기 좋게 인덱스를 1부터 시작하도록 설정
nowon_df_reset.index = range(1, len(nowon_df_reset) + 1)

# -------------------------------
# 자치구 컬럼 추가
# -------------------------------
# 모든 데이터가 노원구이므로 자치구 값을 고정으로 입력
nowon_df_reset['자치구'] = '노원구'

# 컬럼 순서 정렬 (자치구 → 법정동 → 병원수)
nowon_df_reset = nowon_df_reset[['자치구', '법정동', '병원수']]

# -------------------------------
# CSV 파일로 저장
# -------------------------------
# 한글 깨짐 방지를 위해 utf-8-sig 인코딩 사용
nowon_df_reset.to_csv('노원구 동별 병원수.csv', encoding='utf-8-sig')


In [191]:
# ================================
# 강동구 소재 & 영업중인 병원 필터링
# ================================

# 지번주소와 도로명주소 중 값이 있는 것을 하나의 '주소' 컬럼으로 결합
# (주소 정보가 한쪽만 있는 경우를 대비)
hospital_df['주소'] = (
    hospital_df['지번주소'].fillna('') + ' ' +
    hospital_df['도로명주소'].fillna('')
)

# 주소에 '강동구'가 포함되어 있고,
# 병원 영업 상태가 '영업중'인 데이터만 필터링
gangdong_hospital_df = hospital_df[
    hospital_df['주소'].str.contains('강동구', na=False) &
    hospital_df['상세영업상태명'].str.contains('영업중', na=False)
]

# -------------------------------
# 주소에서 자치구(○○구) 추출
# -------------------------------
def gangdong_gu(address):
    """
    주소 문자열을 공백 기준으로 분리한 후,
    '구'로 끝나는 단어를 찾아 자치구 이름을 반환
    """
    parts = address.split()
    for part in parts:
        if part.endswith('구'):
            return part
    return None  # 자치구를 찾지 못한 경우

# 추출한 자치구 정보를 새로운 컬럼으로 추가
gangdong_hospital_df['자치구'] = gangdong_hospital_df['주소'].apply(gangdong_gu)

# 최종 강동구 병원 데이터프레임 확인
gangdong_hospital_df


C:\Users\User\AppData\Local\Temp\ipykernel_11648\322840561.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_df['주소'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_11648\322840561.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gangdong_hospital_df['자치구'] = gangdong_hospital_df['주소'].apply(gangdong_gu)


,상세영업상태명,지번주소,도로명주소,사업장명,주소,자치구
2,영업중,NaN,서울특별시 강동구 천호대로 1044 (성내동),리더스병원,서울특별시 강동구 천호대로 1044 (성내동),강동구
3,영업중,서울특별시 강동구 길동 412-15,"서울특별시 강동구 양재대로 1463, 5~15층 (길동)",강동아너스힐병원,"서울특별시 강동구 길동 412-15 서울특별시 강동구 양재대로 1463, 5~15층...",강동구
27,영업중,NaN,"서울특별시 강동구 강동대로 243, 풍원빌딩 (성내동)",리움한방병원,"서울특별시 강동구 강동대로 243, 풍원빌딩 (성내동)",강동구
28,영업중,"서울특별시 강동구 성내동 452 지하3, 지하1 일부, 1~5층","서울특별시 강동구 강동대로 143-52, 지하3, 지하1 일부, 1~5층 (성내동)",포레스트한방병원,"서울특별시 강동구 성내동 452 지하3, 지하1 일부, 1~5층 서울특별시 강동구 ...",강동구
39,영업중,서울특별시 강동구 성내동 77-27,"서울특별시 강동구 올림픽로 628-3, 지하1, 지상2~9층 (성내동)",123스텝업병원,"서울특별시 강동구 성내동 77-27 서울특별시 강동구 올림픽로 628-3, 지하1,...",강동구
40,영업중,NaN,서울특별시 강동구 천호대로 1151 (길동),고은빛여성병원,서울특별시 강동구 천호대로 1151 (길동),강동구
45,영업중,NaN,"서울특별시 강동구 양재대로 1525, 명품빌딩 (천호동)",올바른서울병원,"서울특별시 강동구 양재대로 1525, 명품빌딩 (천호동)",강동구
47,영업중,서울특별시 강동구 길동 459번지 1호,서울특별시 강동구 천호대로 1133 (길동),남기세병원,서울특별시 강동구 길동 459번지 1호 서울특별시 강동구 천호대로 1133 (길동),강동구
49,영업중,NaN,"서울특별시 강동구 진황도로61길 53, 서울요양병원 (둔촌동)",한국보훈복지의료공단 서울요양병원,"서울특별시 강동구 진황도로61길 53, 서울요양병원 (둔촌동)",강동구
55,영업중,NaN,서울특별시 강동구 양재대로 1580 (명일동),사랑플러스병원,서울특별시 강동구 양재대로 1580 (명일동),강동구


In [192]:
gangdong_hospital_df_count = gangdong_hospital_df['자치구'].value_counts()
gangdong_hospital_df_count

자치구
강동구    38
Name: count, dtype: int64

In [193]:
gangdong_hospital_df['자치구'].unique()

array(['강동구'], dtype=object)

In [194]:
def gangdong_dong_from_addresses(jibun, doromyeong):
    """
    지번주소와 도로명주소를 이용해 법정동(○○동)을 추출하는 함수
    - 두 주소 중 하나라도 동 정보가 있으면 반환
    - '강동구'와 같이 구 이름이 포함된 단어는 제외
    - 찾지 못하면 None 반환
    """

    # 1. 입력값이 문자열이 아닐 경우를 대비해 빈 문자열로 변환
    jibun = jibun if isinstance(jibun, str) else ''
    doromyeong = doromyeong if isinstance(doromyeong, str) else ''
    
    # 2. 지번주소 → 도로명주소 순으로 동 추출 시도
    for address in [jibun, doromyeong]:

        # 2-1. 주소에 괄호가 있는 경우
        #      예: "... (성내동, 강동구)"
        if '(' in address:
            # 괄호 안에서 동 정보만 추출
            inside = (
                address.split('(')[1]
                       .split(')')[0]
                       .split(',')[0]
                       .strip()
            )
            if '동' in inside:
                return inside

        # 2-2. 괄호가 없는 주소인 경우
        #      공백 기준으로 분리 후 '동'이 포함된 단어 탐색
        #      단, '강동구'처럼 '구'가 함께 들어간 단어는 제외
        else:
            for part in address.split():
                if '동' in part and '구' not in part:
                    return part

    # 3. 지번주소와 도로명주소 어디에서도 동을 찾지 못한 경우
    return None


# 각 병원의 지번주소와 도로명주소를 활용해 법정동 컬럼 생성
gangdong_hospital_df['법정동'] = gangdong_hospital_df.apply(
    lambda row: gangdong_dong_from_addresses(row['지번주소'], row['도로명주소']),
    axis=1
)

# 법정동 정보가 추가된 강동구 병원 데이터 확인
gangdong_hospital_df


C:\Users\User\AppData\Local\Temp\ipykernel_11648\4013562810.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gangdong_hospital_df['법정동'] = gangdong_hospital_df.apply(


,상세영업상태명,지번주소,도로명주소,사업장명,주소,자치구,법정동
2,영업중,NaN,서울특별시 강동구 천호대로 1044 (성내동),리더스병원,서울특별시 강동구 천호대로 1044 (성내동),강동구,성내동
3,영업중,서울특별시 강동구 길동 412-15,"서울특별시 강동구 양재대로 1463, 5~15층 (길동)",강동아너스힐병원,"서울특별시 강동구 길동 412-15 서울특별시 강동구 양재대로 1463, 5~15층...",강동구,길동
27,영업중,NaN,"서울특별시 강동구 강동대로 243, 풍원빌딩 (성내동)",리움한방병원,"서울특별시 강동구 강동대로 243, 풍원빌딩 (성내동)",강동구,성내동
28,영업중,"서울특별시 강동구 성내동 452 지하3, 지하1 일부, 1~5층","서울특별시 강동구 강동대로 143-52, 지하3, 지하1 일부, 1~5층 (성내동)",포레스트한방병원,"서울특별시 강동구 성내동 452 지하3, 지하1 일부, 1~5층 서울특별시 강동구 ...",강동구,성내동
39,영업중,서울특별시 강동구 성내동 77-27,"서울특별시 강동구 올림픽로 628-3, 지하1, 지상2~9층 (성내동)",123스텝업병원,"서울특별시 강동구 성내동 77-27 서울특별시 강동구 올림픽로 628-3, 지하1,...",강동구,성내동
40,영업중,NaN,서울특별시 강동구 천호대로 1151 (길동),고은빛여성병원,서울특별시 강동구 천호대로 1151 (길동),강동구,길동
45,영업중,NaN,"서울특별시 강동구 양재대로 1525, 명품빌딩 (천호동)",올바른서울병원,"서울특별시 강동구 양재대로 1525, 명품빌딩 (천호동)",강동구,천호동
47,영업중,서울특별시 강동구 길동 459번지 1호,서울특별시 강동구 천호대로 1133 (길동),남기세병원,서울특별시 강동구 길동 459번지 1호 서울특별시 강동구 천호대로 1133 (길동),강동구,길동
49,영업중,NaN,"서울특별시 강동구 진황도로61길 53, 서울요양병원 (둔촌동)",한국보훈복지의료공단 서울요양병원,"서울특별시 강동구 진황도로61길 53, 서울요양병원 (둔촌동)",강동구,둔촌동
55,영업중,NaN,서울특별시 강동구 양재대로 1580 (명일동),사랑플러스병원,서울특별시 강동구 양재대로 1580 (명일동),강동구,명일동


In [195]:
gangdong_hospital_df['법정동'].unique()

array(['성내동', '길동', '천호동', '둔촌동', '명일동', '상일동', '상일제1동', '고덕동'],
      dtype=object)

In [196]:
# ================================
# 강동구 병원의 동별 개수 집계
# ================================

# 강동구 병원 데이터 중 자치구와 법정동 정보만 추출
total_gangdong_hospital_df = gangdong_hospital_df[['자치구', '법정동']]

# 법정동 기준으로 병원 개수 집계
gangdong_hospital_df_count = total_gangdong_hospital_df['법정동'].value_counts()

# 집계 결과를 데이터프레임 형태로 변환
gangdong_df = pd.DataFrame(gangdong_hospital_df_count)

# 컬럼명 정리 (병원 수 컬럼 이름 변경)
gangdong_df.rename(columns={'count': '병원수'}, inplace=True)

# 인덱스를 컬럼으로 변환
gangdong_df_reset = gangdong_df.reset_index()

# 보기 좋게 인덱스를 1부터 시작하도록 설정
gangdong_df_reset.index = range(1, len(gangdong_df_reset) + 1)

# -------------------------------
# 자치구 컬럼 추가
# -------------------------------
# 모든 데이터가 강동구이므로 자치구 값을 고정으로 입력
gangdong_df_reset['자치구'] = '강동구'

# 컬럼 순서 정렬 (자치구 → 법정동 → 병원수)
gangdong_df_reset = gangdong_df_reset[['자치구', '법정동', '병원수']]

# -------------------------------
# CSV 파일로 저장
# -------------------------------
# 한글 깨짐 방지를 위해 utf-8-sig 인코딩 사용
gangdong_df_reset.to_csv('강동구 동별 병원수.csv', encoding='utf-8-sig')


In [197]:
# ================================
# 성북구 소재 & 영업중인 병원 필터링
# ================================

# 지번주소와 도로명주소 중 값이 있는 것을 하나의 '주소' 컬럼으로 결합
# (주소 정보가 한쪽만 존재하는 경우를 대비)
hospital_df['주소'] = (
    hospital_df['지번주소'].fillna('') + ' ' +
    hospital_df['도로명주소'].fillna('')
)

# 주소에 '성북구'가 포함되어 있고,
# 병원 영업 상태가 '영업중'인 데이터만 필터링
seongbuk_hospital_df = hospital_df[
    hospital_df['주소'].str.contains('성북구', na=False) &
    hospital_df['상세영업상태명'].str.contains('영업중', na=False)
]

# -------------------------------
# 주소에서 자치구(○○구) 추출
# -------------------------------
def seongbuk_gu(address):
    """
    주소 문자열을 공백 기준으로 분리한 후,
    '구'로 끝나는 단어를 찾아 자치구 이름을 반환
    """
    parts = address.split()
    for part in parts:
        if part.endswith('구'):
            return part
    return None  # 자치구를 찾지 못한 경우

# 추출한 자치구 정보를 새로운 컬럼으로 추가
seongbuk_hospital_df['자치구'] = seongbuk_hospital_df['주소'].apply(seongbuk_gu)

# 최종 성북구 병원 데이터프레임 확인
seongbuk_hospital_df


C:\Users\User\AppData\Local\Temp\ipykernel_11648\3368075748.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hospital_df['주소'] = (
C:\Users\User\AppData\Local\Temp\ipykernel_11648\3368075748.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seongbuk_hospital_df['자치구'] = seongbuk_hospital_df['주소'].apply(seongbuk_gu)


,상세영업상태명,지번주소,도로명주소,사업장명,주소,자치구
42,영업중,NaN,서울특별시 성북구 안암로 99 (안암동5가),의료법인 우신향병원,서울특별시 성북구 안암로 99 (안암동5가),성북구
231,영업중,NaN,서울특별시 성북구 화랑로 271 (장위동),의료법인유라의료재단 온누리요양병원,서울특별시 성북구 화랑로 271 (장위동),성북구
232,영업중,NaN,"서울특별시 성북구 종암로 129, 청한빌딩 4,5층 (종암동)",한마음한방병원,"서울특별시 성북구 종암로 129, 청한빌딩 4,5층 (종암동)",성북구
233,영업중,NaN,"서울특별시 성북구 동소문로 106, 유타 8~9층 (동선동1가)",성북미올한방병원,"서울특별시 성북구 동소문로 106, 유타 8~9층 (동선동1가)",성북구
234,영업중,서울특별시 성북구 정릉동 16번지 486호 외2필지,서울특별시 성북구 동소문로 215 (정릉동),새마음요양병원,서울특별시 성북구 정릉동 16번지 486호 외2필지 서울특별시 성북구 동소문로 21...,성북구
235,영업중,NaN,서울특별시 성북구 동소문로47길 8 (길음동),서울척병원,서울특별시 성북구 동소문로47길 8 (길음동),성북구
236,영업중,NaN,서울특별시 성북구 보문로 46 (보문동7가),보문효사랑요양병원,서울특별시 성북구 보문로 46 (보문동7가),성북구
237,영업중,서울특별시 성북구 보문동4가 18번지,서울특별시 성북구 보문로 86 (보문동4가),성북웰니스요양병원,서울특별시 성북구 보문동4가 18번지 서울특별시 성북구 보문로 86 (보문동4가),성북구
238,영업중,NaN,서울특별시 성북구 북악산로1길 71 (정릉동),의료법인참예원의료재단 성북참요양병원,서울특별시 성북구 북악산로1길 71 (정릉동),성북구
239,영업중,서울특별시 성북구 동선동4가 53,"서울특별시 성북구 아리랑로 16, 3~7층 (동선동4가)",더서울병원,"서울특별시 성북구 동선동4가 53 서울특별시 성북구 아리랑로 16, 3~7층 (동선...",성북구


In [198]:
seongbuk_hospital_df_count = seongbuk_hospital_df['자치구'].value_counts()
seongbuk_hospital_df_count

자치구
성북구    16
Name: count, dtype: int64

In [199]:
def seongbuk_dong_from_addresses(jibun, doromyeong):
    """
    지번주소와 도로명주소를 이용해 법정동(○○동)을 추출하는 함수
    - 두 주소 중 하나라도 동 정보가 있으면 반환
    - 괄호 안에 있는 동 정보를 우선적으로 사용
    - 찾지 못하면 None 반환
    """

    # 1. 입력값이 문자열이 아닐 경우 오류 방지를 위해 빈 문자열로 변환
    jibun = jibun if isinstance(jibun, str) else ''
    doromyeong = doromyeong if isinstance(doromyeong, str) else ''
    
    # 2. 지번주소 → 도로명주소 순서로 동 추출 시도
    for address in [jibun, doromyeong]:

        # 2-1. 주소에 괄호가 있는 경우
        #      예: "... (정릉동, 성북구)"
        if '(' in address:
            inside = address.split('(')[1]

            # 닫는 괄호가 있으면 그 이전까지만 사용
            if ')' in inside:
                inside = inside.split(')')[0]

            # 쉼표 앞까지만 추출하여 동 정보만 남김
            inside = inside.split(',')[0].strip()

            # 괄호 안에 '동'이 포함되어 있으면 해당 값 반환
            if '동' in inside:
                return inside

        # 2-2. 괄호가 없는 주소인 경우
        #      공백 기준으로 분리하여 '동'이 포함된 단어 탐색
        else:
            for part in address.split():
                part = part.strip()
                if '동' in part:
                    return part

    # 3. 지번주소와 도로명주소 어디에서도 동을 찾지 못한 경우
    return None


# 각 병원의 지번주소와 도로명주소를 활용해 법정동 컬럼 생성
seongbuk_hospital_df['법정동'] = seongbuk_hospital_df.apply(
    lambda row: seongbuk_dong_from_addresses(row['지번주소'], row['도로명주소']),
    axis=1
)

# 법정동 정보가 추가된 성북구 병원 데이터 확인
seongbuk_hospital_df


C:\Users\User\AppData\Local\Temp\ipykernel_11648\2650907924.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  seongbuk_hospital_df['법정동'] = seongbuk_hospital_df.apply(


,상세영업상태명,지번주소,도로명주소,사업장명,주소,자치구,법정동
42,영업중,NaN,서울특별시 성북구 안암로 99 (안암동5가),의료법인 우신향병원,서울특별시 성북구 안암로 99 (안암동5가),성북구,안암동5가
231,영업중,NaN,서울특별시 성북구 화랑로 271 (장위동),의료법인유라의료재단 온누리요양병원,서울특별시 성북구 화랑로 271 (장위동),성북구,장위동
232,영업중,NaN,"서울특별시 성북구 종암로 129, 청한빌딩 4,5층 (종암동)",한마음한방병원,"서울특별시 성북구 종암로 129, 청한빌딩 4,5층 (종암동)",성북구,종암동
233,영업중,NaN,"서울특별시 성북구 동소문로 106, 유타 8~9층 (동선동1가)",성북미올한방병원,"서울특별시 성북구 동소문로 106, 유타 8~9층 (동선동1가)",성북구,동선동1가
234,영업중,서울특별시 성북구 정릉동 16번지 486호 외2필지,서울특별시 성북구 동소문로 215 (정릉동),새마음요양병원,서울특별시 성북구 정릉동 16번지 486호 외2필지 서울특별시 성북구 동소문로 21...,성북구,정릉동
235,영업중,NaN,서울특별시 성북구 동소문로47길 8 (길음동),서울척병원,서울특별시 성북구 동소문로47길 8 (길음동),성북구,길음동
236,영업중,NaN,서울특별시 성북구 보문로 46 (보문동7가),보문효사랑요양병원,서울특별시 성북구 보문로 46 (보문동7가),성북구,보문동7가
237,영업중,서울특별시 성북구 보문동4가 18번지,서울특별시 성북구 보문로 86 (보문동4가),성북웰니스요양병원,서울특별시 성북구 보문동4가 18번지 서울특별시 성북구 보문로 86 (보문동4가),성북구,보문동4가
238,영업중,NaN,서울특별시 성북구 북악산로1길 71 (정릉동),의료법인참예원의료재단 성북참요양병원,서울특별시 성북구 북악산로1길 71 (정릉동),성북구,정릉동
239,영업중,서울특별시 성북구 동선동4가 53,"서울특별시 성북구 아리랑로 16, 3~7층 (동선동4가)",더서울병원,"서울특별시 성북구 동선동4가 53 서울특별시 성북구 아리랑로 16, 3~7층 (동선...",성북구,동선동4가


In [200]:
# ================================
# 성북구 병원의 동별 개수 집계
# ================================

# 성북구 병원 데이터 중 자치구와 법정동 컬럼만 선택
total_seongbuk_hospital_df = seongbuk_hospital_df[['자치구', '법정동']]

# 법정동 기준으로 병원 개수 집계
seongbuk_hospital_df_count = total_seongbuk_hospital_df['법정동'].value_counts()

# 집계 결과를 데이터프레임 형태로 변환
seongbuk_df = pd.DataFrame(seongbuk_hospital_df_count)

# 컬럼명 정리 (병원 수 컬럼 이름 변경)
seongbuk_df.rename(columns={'count': '병원수'}, inplace=True)

# 인덱스를 컬럼으로 변환
seongbuk_df_reset = seongbuk_df.reset_index()

# 보기 좋게 인덱스를 1부터 시작하도록 설정
seongbuk_df_reset.index = range(1, len(seongbuk_df_reset) + 1)

# -------------------------------
# 자치구 컬럼 추가
# -------------------------------
# 모든 데이터가 성북구이므로 자치구 값을 고정으로 입력
seongbuk_df_reset['자치구'] = '성북구'

# 컬럼 순서 정렬 (자치구 → 법정동 → 병원수)
seongbuk_df_reset = seongbuk_df_reset[['자치구', '법정동', '병원수']]

# -------------------------------
# CSV 파일로 저장
# -------------------------------
# 한글 깨짐 방지를 위해 utf-8-sig 인코딩 사용
seongbuk_df_reset.to_csv('성북구 동별 병원수.csv', encoding='utf-8-sig')


In [202]:
# ================================
# 노원, 강동, 성북 구별 동별 병원 수 통합
# ================================

# 각 데이터프레임에 자치구 컬럼이 이미 있으나,
# 혹시 누락되었을 경우를 대비해 재확인
nowon_df_reset['자치구'] = '노원구'
gangdong_df_reset['자치구'] = '강동구'
seongbuk_df_reset['자치구'] = '성북구'

# 세 구의 데이터프레임을 하나로 합치기
# ignore_index=True: 인덱스를 새로 매김
all_df = pd.concat([nowon_df_reset, gangdong_df_reset, seongbuk_df_reset], ignore_index=True)

# 보기 좋게 1부터 시작하는 번호를 첫 번째 컬럼으로 추가
all_df.insert(0, '', range(1, len(all_df) + 1))

# CSV 파일로 저장
# encoding='utf-8-sig': 한글 깨짐 방지
# index=False: 엑셀에서 불필요한 인덱스 열 생성 방지
all_df.to_csv('최종_노원구_강동구_성북구_병원데이터.csv', encoding='utf-8-sig', index=False)
